In [1]:
import wandb
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
########################################################################################
# query data from wandb

# add data to frames
api = wandb.Api()

data_description_map = {
    "tiny-few": "tiny-few (50k files shared between 100 speakers)",
    "tiny-high": "tiny-high (8 files from 8 sessions for 5994 speakers)",
}


def load_runs(name: str):
    runs = api.runs(name)
    df = pd.DataFrame(columns=["eer", "data", "ablation"])

    for r in runs:
        tags = r.tags

        if r.state != "finished":
            continue

        eer = r.summary["test_eer_hard"]

        if "tiny_few" in tags:
            tags.remove("tiny_few")
            data = "tiny_few"
        elif "tiny_many_high" in tags:
            tags.remove("tiny_many_high")
            data = "tiny_many_high"
        else:
            raise ValueError(f"undetermined dataset from {tags=}")

        ablation = tags[0]

        df = pd.concat(
            [
                df,
                pd.DataFrame(
                    {
                        "ablation": [ablation],
                        "eer": [eer],
                        "data": [data],
                    }
                ),
            ],
            ignore_index=True,
        )

    return df


df = load_runs("wav2vec2-ablation")

In [3]:
df

,eer,data,ablation
0,0.077306,tiny_many_high,reg_mask
1,0.08045,tiny_many_high,reg_dropout
2,0.06733,tiny_many_high,reg_layerdrop
3,0.462065,tiny_many_high,weights_random_init
4,0.077615,tiny_many_high,reg_none
...,...,...,...
68,0.067629,tiny_many_high,baseline
69,0.168396,tiny_few,weights_xlsr
70,0.066954,tiny_many_high,weights_xlsr
71,0.14854,tiny_few,baseline


In [4]:
df_grouped = df.groupby(by=["data", "ablation",])

In [5]:
df_agg = df_grouped.agg(
    eer_min=("eer", "min"),
    eer_max=("eer", "max"),
    eer_mean=("eer", "mean"),
    eer_std=("eer", "std"),
    count=("eer", "count")
)

In [6]:
df_agg

eer_min   eer_max  eer_mean  \
data           ablation                                                 
tiny_few       baseline                  0.148540  0.151224  0.150236   
               lr_1_cycle                0.155269  0.159669  0.157871   
               lr_constant               0.165800  0.170624  0.167673   
               lr_exp_decay              0.165158  0.169016  0.166832   
               reg_dropout               0.165495  0.167647  0.166676   
               reg_layerdrop             0.148761  0.151634  0.150259   
               reg_mask                  0.160021  0.163214  0.162132   
               reg_none                  0.163649  0.168333  0.166722   
               weights_freeze_cycle      0.143980  0.145876  0.145224   
               weights_freeze_cycle_cnn  0.152798  0.157593  0.154793   
               weights_no_freeze         0.149604  0.152833  0.151554   
               weights_random_init       0.332113  0.335212  0.333499   
               weights_xlsr              0.168396  0.168396  0.168396   
tiny_many_high baseline                  0.066402  0.069767  0.067933   
               lr_1_cycle                0.083266  0.087474  0.085880   
               lr_constant               0.085005  0.093064  0.087955   
               lr_exp_decay              0.066222  0.066994  0.066665   
               reg_dropout               0.078807  0.081169  0.080142   
               reg_layerdrop             0.065081  0.069292  0.067234   
               reg_mask                  0.077306  0.081509  0.078728   
               reg_none                  0.076413  0.077765  0.077265   
               weights_freeze_cycle      0.061858  0.065135  0.063829   
               weights_freeze_cycle_cnn  0.074269  0.083212  0.078287   
               weights_no_freeze         0.070450  0.073201  0.071825   
               weights_random_init       0.462009  0.463056  0.462376   
               weights_xlsr              0.066954  0.066954  0.066954   

                                          eer_std  count  
data           ablation                                   
tiny_few       baseline                  0.001475      3  
               lr_1_cycle                0.002308      3  
               lr_constant               0.002586      3  
               lr_exp_decay              0.001979      3  
               reg_dropout               0.001091      3  
               reg_layerdrop             0.001440      3  
               reg_mask                  0.001828      3  
               reg_none                  0.002662      3  
               weights_freeze_cycle      0.001078      3  
               weights_freeze_cycle_cnn  0.002497      3  
               weights_no_freeze         0.001716      3  
               weights_random_init       0.001575      3  
               weights_xlsr                   NaN      1  
tiny_many_high baseline                  0.001703      3  
               lr_1_cycle                0.002282      3  
               lr_constant               0.004442      3  
               lr_exp_decay              0.000398      3  
               reg_dropout               0.001210      3  
               reg_layerdrop             0.002107      3  
               reg_mask                  0.002408      3  
               reg_none                  0.000741      3  
               weights_freeze_cycle      0.001737      3  
               weights_freeze_cycle_cnn  0.004540      3  
               weights_no_freeze         0.001946      2  
               weights_random_init       0.000589      3  
               weights_xlsr                   NaN      1

In [7]:
df_agg['count'].sum()

73